In [1]:
import numpy as np

import MDAnalysis as mda
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

from MDAnalysis.analysis import rms

import os
import glob


/home/user/anaconda3/envs/Xponge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/anaconda3/envs/Xponge/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [2]:

aligned_traj = mda.Universe('8qw7_A.pdb', 'aligned.pdb')

POI_ref = mda.Universe('8qw7_A.pdb', '8qw7_A.pdb')

E3_ref = mda.Universe('8qw7_B.pdb', '8qw7_B.pdb')

aligned_traj_CA = aligned_traj.select_atoms('name CA')

POI_ref_CA = POI_ref.select_atoms('name CA')


In [3]:

print(len(aligned_traj.trajectory))

rmsd_traj = []
for ts in aligned_traj.trajectory:
    r = rms.rmsd(aligned_traj_CA.positions, POI_ref_CA.positions, center=False, superposition=False)
    rmsd_traj.append(r)
print(len(rmsd_traj))
print(rmsd_traj)


33120
33120
[36.45996310971196, 36.93969639829593, 36.52286214602559, 36.0375833275069, 36.0016257004771, 35.57380885420394, 37.02879912979511, 36.128915535968304, 35.63827043756397, 36.35204460484707, 36.11014476566362, 35.565491574798706, 36.539774725771736, 36.640289521974374, 36.062921474737195, 35.836564430876436, 35.68849492861567, 35.627550612406246, 35.169229585199744, 35.75860061737591, 36.579527897198716, 35.26280647248605, 35.60777523989621, 36.039559214651014, 35.7033662698609, 35.132374241936525, 35.16676612892189, 36.52050046780953, 36.27519158419828, 36.99145148410074, 35.215187751606805, 35.34186807398543, 37.55454071987488, 35.51954012044374, 34.788699894053, 36.07112518707417, 34.7226491736669, 35.22011342358619, 35.412660341399864, 35.17516886862521, 37.17152282347473, 36.29655666887192, 34.91195390020289, 34.74893635237693, 34.72867593979203, 35.752323353235965, 35.93422239858126, 35.644138516300224, 35.020374026440244, 36.81169823952605, 34.71414026097432, 34.43301

In [4]:
clustering_method = clm.AffinityPropagationNative(preference=-1.0,
                                                  damping=0.9,
                                                  max_iter=200,
                                                  convergence_iter=30,
                                                  add_noise=True)

km1 = clm.KMeans(100,
                 init = 'k-means++',
                 algorithm="auto")

ces0, details0 = encore.ces([aligned_traj],
                            select='name CA',
                            clustering_method=km1,
                            ncores=10)


/home/user/anaconda3/envs/Xponge/lib/python3.12/site-packages/MDAnalysis/core/universe.py:658: UserWarning: Reader has no dt information, set to 1.0 ps
  dt=self.trajectory.ts.dt * step,


In [5]:

cluster_collection = details0['clustering'][0]
print(type(cluster_collection))
print('We have found {} clusters'.format(len(cluster_collection)))

<class 'MDAnalysis.analysis.encore.clustering.ClusterCollection.ClusterCollection'>
We have found 100 clusters


In [6]:
first_cluster = cluster_collection.clusters[0]
first_cluster


<Cluster with 342 elements, centroid=44, id=0>

In [7]:
first_cluster.elements

array([    0,     1,     2,     3,     4,     5,     6,     7,     8,
          10,    11,    12,    13,    14,    15,    16,    17,    18,
          19,    21,    22,    24,    25,    26,    27,    28,    29,
          30,    31,    32,    34,    35,    36,    37,    38,    39,
          40,    42,    43,    44,    47,    49,    50,    51,    53,
          54,    55,    56,    57,    58,    60,    62,    63,    65,
          68,    69,    70,    72,    74,    75,    77,    78,    79,
          81,    82,    83,    84,    85,    86,    87,    88,    89,
          90,    91,    95,    97,    98,    99,   102,   104,   107,
         108,   109,   111,   112,   113,   115,   116,   118,   119,
         120,   121,   123,   124,   127,   133,   134,   137,   138,
         139,   141,   143,   144,   147,   148,   158,   160,   162,
         163,   164,   167,   170,   179,   180,   187,   192,   193,
         200,   204,   206,   210,   220,   225,   229,   231,   236,
         244,   245,

In [8]:
os.mkdir('clusters')


In [9]:

ag = aligned_traj.select_atoms("all")

with open('./clusters/cluster.log', 'w') as cluster_output:
    for i_cluster in cluster_collection:
        print(i_cluster)
        cluster_output.write(str(i_cluster))
        print(i_cluster.id)
        print(i_cluster.size)
        print(i_cluster.centroid)
        print(i_cluster.elements)
        cluster_output.write(',')
        cluster_output.write(np.array2string(i_cluster.elements, max_line_width=100000, precision=0, separator=' ',suppress_small=True))
        cluster_output.write('\n')

        i_cluster_elements = [i_cluster.centroid]
        ag.write('./clusters/cluster_' + str(i_cluster.id) + '.pdb', frames=aligned_traj.trajectory[i_cluster_elements])
        POI_clustered = mda.Universe('./clusters/cluster_' + str(i_cluster.id) + '.pdb', './clusters/cluster_' + str(i_cluster.id) + '.pdb')
        POI_and_E3 = mda.Merge(POI_clustered.atoms, E3_ref.atoms)
        POI_and_E3.atoms.write('./clusters/POI_and_E3_cluster_' + str(i_cluster.id) + '.pdb')
        for i_cluster_element in i_cluster.elements:
            if i_cluster_element == i_cluster.centroid:
                continue
            i_cluster_elements.append(i_cluster_element)

        print(i_cluster_elements)
        #ag.write('./clusters/cluster_' + str(i_cluster.id) + '.pdb', frames=aligned_traj.trajectory[i_cluster_elements])


rmsd_traj = []
for i_cluster in cluster_collection:
    cluster_traj = mda.Universe('8qw7_A.pdb', './clusters/cluster_' + str(i_cluster.id) + '.pdb')
    cluster_traj_CA = cluster_traj.select_atoms("name CA")
    r = rms.rmsd(cluster_traj_CA.positions, POI_ref_CA.positions, center=False, superposition=False)
    rmsd_traj.append(r)

print('rmsd_traj:')
print(rmsd_traj)



<Cluster with 342 elements, centroid=44, id=0>
0
342
44
[    0     1     2     3     4     5     6     7     8    10    11    12
    13    14    15    16    17    18    19    21    22    24    25    26
    27    28    29    30    31    32    34    35    36    37    38    39
    40    42    43    44    47    49    50    51    53    54    55    56
    57    58    60    62    63    65    68    69    70    72    74    75
    77    78    79    81    82    83    84    85    86    87    88    89
    90    91    95    97    98    99   102   104   107   108   109   111
   112   113   115   116   118   119   120   121   123   124   127   133
   134   137   138   139   141   143   144   147   148   158   160   162
   163   164   167   170   179   180   187   192   193   200   204   206
   210   220   225   229   231   236   244   245   249   253   257   258
   261   265   271   282   294   305   306   308   310   320   326   331
   351   359   360   369   382   417   423   440   448   463   472  

/home/user/anaconda3/envs/Xponge/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/user/anaconda3/envs/Xponge/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[843, 145, 151, 166, 175, 177, 181, 183, 185, 189, 190, 196, 197, 199, 203, 207, 208, 212, 213, 216, 221, 232, 234, 243, 247, 248, 250, 251, 252, 255, 256, 259, 267, 268, 270, 275, 276, 280, 283, 284, 285, 288, 289, 291, 293, 295, 296, 297, 299, 303, 312, 314, 316, 324, 325, 327, 332, 335, 336, 339, 340, 342, 344, 346, 348, 349, 355, 358, 361, 362, 363, 364, 367, 368, 374, 387, 388, 389, 390, 391, 393, 399, 401, 405, 407, 408, 410, 412, 415, 418, 421, 429, 431, 432, 433, 436, 441, 443, 450, 454, 455, 457, 460, 461, 462, 466, 484, 486, 487, 489, 494, 497, 498, 501, 514, 516, 532, 546, 548, 549, 550, 559, 561, 584, 585, 588, 612, 613, 652, 654, 662, 670, 700, 708, 713, 735, 744, 747, 762, 788, 798, 805, 829, 841, 846, 851, 854, 875, 878, 883, 889, 902, 910, 911, 933, 970, 978, 980, 985, 991, 1006, 1024, 1051, 1073, 1076, 1080, 1086, 1099, 1103, 1109, 1115, 1146, 1149, 1166, 1178, 1185, 1198, 1207, 1209, 1223, 1234, 1250, 1260, 1276, 1313, 1319, 1326, 1342, 1347, 1361, 1368, 1407, 1408, 1